# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Import NHL data
nhl_df=pd.read_csv("assets/nhl.csv")

# Modify NHL dataframe
nhl_df=nhl_df[nhl_df['year']==2018] # select year 2018
nhl_df.replace({'\*':''},inplace=True, regex=True) # remove all *
nhl_df.drop(['OL','PTS','PTS%','GF','GA', 'SRS','SOS','RPt%','ROW'],axis=1, inplace=True) # drop columns we don't use
nhl_df['location'] = nhl_df['team'].map({'Tampa Bay Lightning':'Tampa Bay Area', # map all teams
                                           'Boston Bruins':'Boston',
                                           'Toronto Maple Leafs':'Toronto',
                                           'Florida Panthers':'Miami–Fort Lauderdale',
                                           'Detroit Red Wings':'Detroit',
                                           'Montreal Canadiens':'Montreal',
                                           'Ottawa Senators':'Ottawa',
                                           'Buffalo Sabres':'Buffalo',
                                           'Washington Capitals':'Washington, D.C.',
                                           'Pittsburgh Penguins':'Pittsburgh',
                                           'Philadelphia Flyers':'Philadelphia',
                                           'Columbus Blue Jackets':'Columbus',
                                           'New Jersey Devils':'New York City',
                                           'Carolina Hurricanes':'Raleigh',
                                           'New York Islanders':'New York City',
                                           'New York Rangers':'New York City',
                                           'Nashville Predators':'Nashville',
                                           'Winnipeg Jets':'Winnipeg',
                                           'Minnesota Wild':'Minneapolis–Saint Paul',
                                           'Colorado Avalanche':'Denver',
                                           'St. Louis Blues':'St. Louis',
                                           'Dallas Stars':'Dallas–Fort Worth',
                                           'Chicago Blackhawks':'Chicago',
                                           'Vegas Golden Knights':'Las Vegas',
                                           'Anaheim Ducks':'Los Angeles',
                                           'San Jose Sharks':'San Francisco Bay Area',
                                           'Los Angeles Kings':'Los Angeles',
                                           'Calgary Flames':'Calgary',
                                           'Edmonton Oilers':'Edmonton',
                                           'Vancouver Canucks':'Vancouver',
                                           'Arizona Coyotes':'Phoenix'})
     

# Import cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# modify city dataframe
cities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
cities.replace({'\[.*\]':''},inplace=True, regex=True) # remove all square blanket
cities.replace(['','— ','—'], np.nan, inplace=True) # label missing values


def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # merge the dataframe
    df = pd.merge(nhl_df, cities, how='inner', left_on='location', right_on='Metropolitan area')
    
    # modify dataframe
    df.drop(['NFL','MLB','NBA','NHL'], axis=1, inplace=True) # drop useless columns
    df = df.astype({'W':int,'L':int, 'Population':int}) # change str to int
    df['W/L'] = df['W']/(df['W']+df['L'])
        
    population_by_region = df.groupby('location').agg({'W/L':np.nanmean, 'Population':np.nanmean}).iloc[:,1] # pass in metropolitan area population from cities
    win_loss_by_region = df.groupby('location').agg({'W/L':np.nanmean, 'Population':np.nanmean}).iloc[:,0] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Import NBA data
nba_df=pd.read_csv("assets/nba.csv")

# Modify NBA dataframe
nba_df=nba_df[nba_df['year']==2018] # select year 2018
nba_df.drop(['GB','PS/G','PA/G','SRS', 'W', 'L'],axis=1, inplace=True) # drop columns we don't use
nba_df.replace({'\(.*\)':'', '\*':'', '\xa0':''},inplace=True, regex=True) # remove all useless char
nba_df['location']=nba_df['team'].map({'Toronto Raptors':'Toronto', # Map team locations
                                         'Boston Celtics':'Boston',
                                         'Philadelphia 76ers':'Philadelphia',
                                         'Cleveland Cavaliers':'Cleveland',
                                         'Indiana Pacers':'Indianapolis',
                                         'Miami Heat':'Miami–Fort Lauderdale',
                                         'Milwaukee Bucks':'Milwaukee',
                                         'Washington Wizards':'Washington, D.C.',
                                         'Detroit Pistons':'Detroit',
                                         'Charlotte Hornets':'Charlotte',
                                         'New York Knicks':'New York City',
                                         'Brooklyn Nets':'New York City',
                                         'Chicago Bulls':'Chicago',
                                         'Orlando Magic':'Orlando',
                                         'Atlanta Hawks':'Atlanta',
                                         'Houston Rockets':'Houston',
                                         'Golden State Warriors':'San Francisco Bay Area',
                                         'Portland Trail Blazers':'Portland',
                                         'Oklahoma City Thunder':'Oklahoma City',
                                         'Utah Jazz':'Salt Lake City',
                                         'New Orleans Pelicans':'New Orleans',
                                         'San Antonio Spurs':'San Antonio',
                                         'Minnesota Timberwolves':'Minneapolis–Saint Paul',
                                         'Denver Nuggets':'Denver',
                                         'Los Angeles Clippers':'Los Angeles',
                                         'Los Angeles Lakers':'Los Angeles',
                                         'Sacramento Kings':'Sacramento',
                                         'Dallas Mavericks':'Dallas–Fort Worth',
                                         'Memphis Grizzlies':'Memphis',
                                         'Phoenix Suns':'Phoenix'})

# Import cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# Modify cities
cities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
cities.replace({'\[.*\]':''},inplace=True, regex=True) # remove all square blanket
cities.replace(['','— ','—'], np.nan, inplace=True) # label missing values

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # merge NBA and cities
    df = pd.merge(nba_df, cities, how='inner', left_on='location', right_on='Metropolitan area')
    
    # modify dataframe
    df.drop(['NFL','MLB','NBA','NHL'], axis=1, inplace=True) # drop useless columns
    df = df.astype({'W/L%':float, 'Population':int}) # change str to int
    
    population_by_region = df.groupby('location').agg({'W/L%':np.nanmean,'Population':np.nanmean}).iloc[:,1] # pass in metropolitan area population from cities
    win_loss_by_region = df.groupby('location').agg({'W/L%':np.nanmean,'Population':np.nanmean}).iloc[:,0] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Import NBA data
mlb_df=pd.read_csv("assets/mlb.csv")

# Modify NBA dataframe
mlb_df=mlb_df[mlb_df['year']==2018] # select year 2018
mlb_df.drop(['W', 'L'],axis=1, inplace=True) # drop columns we don't use
mlb_df['location'] = mlb_df['team'].map({'Boston Red Sox':'Boston', # map all team locations
                                         'New York Yankees':'New York City',
                                         'Tampa Bay Rays':'Tampa Bay Area',
                                         'Toronto Blue Jays':'Toronto',
                                         'Baltimore Orioles':'Baltimore',
                                         'Cleveland Indians':'Cleveland',
                                         'Minnesota Twins':'Minneapolis–Saint Paul',
                                         'Detroit Tigers':'Detroit',
                                         'Chicago White Sox':'Chicago',
                                         'Kansas City Royals':'Kansas City',
                                         'Houston Astros':'Houston',
                                         'Oakland Athletics':'San Francisco Bay Area',
                                         'Seattle Mariners':'Seattle',
                                         'Los Angeles Angels':'Los Angeles',
                                         'Texas Rangers':'Dallas–Fort Worth',
                                         'Atlanta Braves':'Atlanta',
                                         'Washington Nationals':'Washington, D.C.',
                                         'Philadelphia Phillies':'Philadelphia',
                                         'New York Mets':'New York City',
                                         'Miami Marlins':'Miami–Fort Lauderdale',
                                         'Milwaukee Brewers':'Milwaukee',
                                         'Chicago Cubs':'Chicago',
                                         'St. Louis Cardinals':'St. Louis',
                                         'Pittsburgh Pirates':'Pittsburgh',
                                         'Cincinnati Reds':'Cincinnati',
                                         'Los Angeles Dodgers':'Los Angeles',
                                         'Colorado Rockies':'Denver',
                                         'Arizona Diamondbacks':'Phoenix',
                                         'San Francisco Giants':'San Francisco Bay Area',
                                         'San Diego Padres':'San Diego'})

# Import cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# Modify cities
cities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
cities.replace({'\[.*\]':''},inplace=True, regex=True) # remove all square blanket
cities.replace(['','— ','—'], np.nan, inplace=True) # label missing values

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # merge dataframe
    df = pd.merge(mlb_df, cities, how='inner', left_on='location', right_on='Metropolitan area')
    
    # modify dataframe
    df.drop(['NFL','MLB','NBA','NHL'], axis=1, inplace=True) # drop useless columns
    df = df.astype({'W-L%':float, 'Population':int}) # change str to int
    
    population_by_region = df.groupby('location').agg({'W-L%':np.nanmean, 'Population':np.nanmean}).iloc[:,1] # pass in metropolitan area population from cities
    win_loss_by_region = df.groupby('location').agg({'W-L%':np.nanmean, 'Population':np.nanmean}).iloc[:,0] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Import NFL data
nfl_df=pd.read_csv("assets/nfl.csv")

# Modify NFL dataframe
nfl_df=nfl_df[nfl_df['year']==2018]
nfl_df.replace({'\*':'', '\+':''},inplace=True, regex=True) # remove all useless char
nfl_df.drop(['DSRS','L','League','MoV','OSRS','PA','PD','PF','SRS','SoS','T','W'], axis=1, inplace=True) # drop useless columns
nfl_df['location'] = nfl_df['team'].map({'New England Patriots':'Boston', # map all teams
                                         'Miami Dolphins':'Miami–Fort Lauderdale',
                                         'Buffalo Bills':'Buffalo',
                                         'New York Jets':'New York City',
                                         'Baltimore Ravens':'Baltimore',
                                         'Pittsburgh Steelers':'Pittsburgh',
                                         'Cleveland Browns':'Cleveland',
                                         'Cincinnati Bengals':'Cincinnati',
                                         'Houston Texans':'Houston',
                                         'Indianapolis Colts':'Indianapolis',
                                         'Tennessee Titans':'Nashville',
                                         'Jacksonville Jaguars':'Jacksonville',
                                         'Kansas City Chiefs':'Kansas City',
                                         'Los Angeles Chargers':'Los Angeles',
                                         'Denver Broncos':'Denver',
                                         'Oakland Raiders':'San Francisco Bay Area',
                                         'Dallas Cowboys':'Dallas–Fort Worth',
                                         'Philadelphia Eagles':'Philadelphia',
                                         'Washington Redskins':'Washington, D.C.',
                                         'New York Giants':'New York City',
                                         'Chicago Bears':'Chicago',
                                         'Minnesota Vikings':'Minneapolis–Saint Paul',
                                         'Green Bay Packers':'Green Bay',
                                         'Detroit Lions':'Detroit',
                                         'New Orleans Saints':'New Orleans',
                                         'Carolina Panthers':'Charlotte',
                                         'Atlanta Falcons':'Atlanta',
                                         'Tampa Bay Buccaneers':'Tampa Bay Area',
                                         'Los Angeles Rams':'Los Angeles',
                                         'Seattle Seahawks':'Seattle',
                                         'San Francisco 49ers':'San Francisco Bay Area',
                                         'Arizona Cardinals':'Phoenix'}) 

# Import cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# Modify cities
cities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
cities.replace({'\[.*\]':''},inplace=True, regex=True) # remove all square blanket
cities.replace(['','— ','—'], np.nan, inplace=True) # label missing values

def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # merge dataframe
    df=pd.merge(nfl_df, cities, how='inner', left_on='location', right_on='Metropolitan area')
    
    # modify dataframe
    df.drop(['NFL','MLB','NBA','NHL'], axis=1, inplace=True) # drop useless columns
    df=df.astype({'W-L%':float, 'Population':int}) # change str to float and int

    population_by_region = df.groupby('location').agg({'W-L%':np.nanmean,'Population':np.nanmean}).iloc[:,1] # pass in metropolitan area population from cities
    win_loss_by_region = df.groupby('location').agg({'W-L%':np.nanmean,'Population':np.nanmean}).iloc[:,0] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [76]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

# Import NBA data
mlb_df=pd.read_csv("assets/mlb.csv")

# Modify NBA dataframe
mlb_df=mlb_df[mlb_df['year']==2018] # select year 2018
mlb_df.drop(['W', 'L','GB','year','League'],axis=1, inplace=True) # drop columns we don't use
mlb_df.rename(columns={'W-L%':'W/L'}, inplace=True) # Unify to name W/L
mlb_df['location'] = mlb_df['team'].map({'Boston Red Sox':'Boston', # map all team locations
                                         'New York Yankees':'New York City',
                                         'Tampa Bay Rays':'Tampa Bay Area',
                                         'Toronto Blue Jays':'Toronto',
                                         'Baltimore Orioles':'Baltimore',
                                         'Cleveland Indians':'Cleveland',
                                         'Minnesota Twins':'Minneapolis–Saint Paul',
                                         'Detroit Tigers':'Detroit',
                                         'Chicago White Sox':'Chicago',
                                         'Kansas City Royals':'Kansas City',
                                         'Houston Astros':'Houston',
                                         'Oakland Athletics':'San Francisco Bay Area',
                                         'Seattle Mariners':'Seattle',
                                         'Los Angeles Angels':'Los Angeles',
                                         'Texas Rangers':'Dallas–Fort Worth',
                                         'Atlanta Braves':'Atlanta',
                                         'Washington Nationals':'Washington, D.C.',
                                         'Philadelphia Phillies':'Philadelphia',
                                         'New York Mets':'New York City',
                                         'Miami Marlins':'Miami–Fort Lauderdale',
                                         'Milwaukee Brewers':'Milwaukee',
                                         'Chicago Cubs':'Chicago',
                                         'St. Louis Cardinals':'St. Louis',
                                         'Pittsburgh Pirates':'Pittsburgh',
                                         'Cincinnati Reds':'Cincinnati',
                                         'Los Angeles Dodgers':'Los Angeles',
                                         'Colorado Rockies':'Denver',
                                         'Arizona Diamondbacks':'Phoenix',
                                         'San Francisco Giants':'San Francisco Bay Area',
                                         'San Diego Padres':'San Diego'})

# Import NHL data
nhl_df=pd.read_csv("assets/nhl.csv")

# Modify NHL dataframe
nhl_df=nhl_df[nhl_df['year']==2018] # select year 2018
nhl_df.replace({'\*':''},inplace=True, regex=True) # remove all *
nhl_df=nhl_df[~nhl_df.team.str.match('.*Division')] # remove division row
nhl_df=nhl_df.astype({'W':int,'L':int}) # change dtype for calculation
nhl_df['W/L']=nhl_df['W']/(nhl_df['W']+nhl_df['L']) # calculate W/L
nhl_df.drop(['OL','PTS','PTS%','GF','GA', 'SRS','SOS','RPt%','ROW','W','L','GP','year','League'],axis=1, inplace=True) # drop columns we don't use
nhl_df['location'] = nhl_df['team'].map({'Tampa Bay Lightning':'Tampa Bay Area', # map all teams
                                           'Boston Bruins':'Boston',
                                           'Toronto Maple Leafs':'Toronto',
                                           'Florida Panthers':'Miami–Fort Lauderdale',
                                           'Detroit Red Wings':'Detroit',
                                           'Montreal Canadiens':'Montreal',
                                           'Ottawa Senators':'Ottawa',
                                           'Buffalo Sabres':'Buffalo',
                                           'Washington Capitals':'Washington, D.C.',
                                           'Pittsburgh Penguins':'Pittsburgh',
                                           'Philadelphia Flyers':'Philadelphia',
                                           'Columbus Blue Jackets':'Columbus',
                                           'New Jersey Devils':'New York City',
                                           'Carolina Hurricanes':'Raleigh',
                                           'New York Islanders':'New York City',
                                           'New York Rangers':'New York City',
                                           'Nashville Predators':'Nashville',
                                           'Winnipeg Jets':'Winnipeg',
                                           'Minnesota Wild':'Minneapolis–Saint Paul',
                                           'Colorado Avalanche':'Denver',
                                           'St. Louis Blues':'St. Louis',
                                           'Dallas Stars':'Dallas–Fort Worth',
                                           'Chicago Blackhawks':'Chicago',
                                           'Vegas Golden Knights':'Las Vegas',
                                           'Anaheim Ducks':'Los Angeles',
                                           'San Jose Sharks':'San Francisco Bay Area',
                                           'Los Angeles Kings':'Los Angeles',
                                           'Calgary Flames':'Calgary',
                                           'Edmonton Oilers':'Edmonton',
                                           'Vancouver Canucks':'Vancouver',
                                           'Arizona Coyotes':'Phoenix'})

# Import NBA data
nba_df=pd.read_csv("assets/nba.csv")

# Modify NBA dataframe
nba_df=nba_df[nba_df['year']==2018] # select year 2018
nba_df.rename(columns={'W/L%':'W/L'}, inplace=True) # Unify to name W/L
nba_df.drop(['GB','PS/G','PA/G','SRS', 'W', 'L','year','League'],axis=1, inplace=True) # drop columns we don't use
nba_df.replace({'\(.*\)':'', '\*':'', '\xa0':''},inplace=True, regex=True) # remove all useless char
nba_df=nba_df.astype({'W/L':float}) # change dtype for calculation
nba_df['location']=nba_df['team'].map({'Toronto Raptors':'Toronto', # Map team locations
                                         'Boston Celtics':'Boston',
                                         'Philadelphia 76ers':'Philadelphia',
                                         'Cleveland Cavaliers':'Cleveland',
                                         'Indiana Pacers':'Indianapolis',
                                         'Miami Heat':'Miami–Fort Lauderdale',
                                         'Milwaukee Bucks':'Milwaukee',
                                         'Washington Wizards':'Washington, D.C.',
                                         'Detroit Pistons':'Detroit',
                                         'Charlotte Hornets':'Charlotte',
                                         'New York Knicks':'New York City',
                                         'Brooklyn Nets':'New York City',
                                         'Chicago Bulls':'Chicago',
                                         'Orlando Magic':'Orlando',
                                         'Atlanta Hawks':'Atlanta',
                                         'Houston Rockets':'Houston',
                                         'Golden State Warriors':'San Francisco Bay Area',
                                         'Portland Trail Blazers':'Portland',
                                         'Oklahoma City Thunder':'Oklahoma City',
                                         'Utah Jazz':'Salt Lake City',
                                         'New Orleans Pelicans':'New Orleans',
                                         'San Antonio Spurs':'San Antonio',
                                         'Minnesota Timberwolves':'Minneapolis–Saint Paul',
                                         'Denver Nuggets':'Denver',
                                         'Los Angeles Clippers':'Los Angeles',
                                         'Los Angeles Lakers':'Los Angeles',
                                         'Sacramento Kings':'Sacramento',
                                         'Dallas Mavericks':'Dallas–Fort Worth',
                                         'Memphis Grizzlies':'Memphis',
                                         'Phoenix Suns':'Phoenix'})

# Import NFL data
nfl_df=pd.read_csv("assets/nfl.csv")

# Modify NFL dataframe
nfl_df=nfl_df[nfl_df['year']==2018] # select year 2018
nfl_df.rename(columns={'W-L%':'W/L'}, inplace=True) # Unify to name W/L
nfl_df.replace({'\*':'', '\+':''},inplace=True, regex=True) # remove all useless char
nfl_df=nfl_df[~nfl_df.team.str.match('[AN]FC')] # remove NFC AFC row
nfl_df.drop(['DSRS','L','League','MoV','OSRS','PA','PD','PF','SRS','SoS','T','W','year'], axis=1, inplace=True) # drop useless columns
nfl_df=nfl_df.astype({'W/L':float}) # change dtype for calculation
nfl_df['location'] = nfl_df['team'].map({'New England Patriots':'Boston', # map all teams
                                         'Miami Dolphins':'Miami–Fort Lauderdale',
                                         'Buffalo Bills':'Buffalo',
                                         'New York Jets':'New York City',
                                         'Baltimore Ravens':'Baltimore',
                                         'Pittsburgh Steelers':'Pittsburgh',
                                         'Cleveland Browns':'Cleveland',
                                         'Cincinnati Bengals':'Cincinnati',
                                         'Houston Texans':'Houston',
                                         'Indianapolis Colts':'Indianapolis',
                                         'Tennessee Titans':'Nashville',
                                         'Jacksonville Jaguars':'Jacksonville',
                                         'Kansas City Chiefs':'Kansas City',
                                         'Los Angeles Chargers':'Los Angeles',
                                         'Denver Broncos':'Denver',
                                         'Oakland Raiders':'San Francisco Bay Area',
                                         'Dallas Cowboys':'Dallas–Fort Worth',
                                         'Philadelphia Eagles':'Philadelphia',
                                         'Washington Redskins':'Washington, D.C.',
                                         'New York Giants':'New York City',
                                         'Chicago Bears':'Chicago',
                                         'Minnesota Vikings':'Minneapolis–Saint Paul',
                                         'Green Bay Packers':'Green Bay',
                                         'Detroit Lions':'Detroit',
                                         'New Orleans Saints':'New Orleans',
                                         'Carolina Panthers':'Charlotte',
                                         'Atlanta Falcons':'Atlanta',
                                         'Tampa Bay Buccaneers':'Tampa Bay Area',
                                         'Los Angeles Rams':'Los Angeles',
                                         'Seattle Seahawks':'Seattle',
                                         'San Francisco 49ers':'San Francisco Bay Area',
                                         'Arizona Cardinals':'Phoenix'})

# Import cities
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# Modify cities
cities.rename(columns = {'Population (2016 est.)[8]':'Population'}, inplace = True)
cities.replace({'\[.*\]':''},inplace=True, regex=True) # remove all square blanket
cities.replace(['','— ','—'], np.nan, inplace=True) # label missing values
cities.drop(['NFL','MLB','NBA','NHL'], axis=1, inplace=True) # drop useless columns
cities = cities.astype({'Population':int}) # change dtpye for calculation

def sport_merger(s): # use function to automate the dataframe merge and groupby
    if s == 'NFL':
        return pd.merge(cities, nfl_df, how='inner', left_on='Metropolitan area', right_on='location').groupby('Metropolitan area').agg({'W/L':np.nanmean,'Population':np.nanmean})
    elif s == 'NBA':
        return pd.merge(cities, nba_df, how='inner', left_on='Metropolitan area', right_on='location').groupby('Metropolitan area').agg({'W/L':np.nanmean,'Population':np.nanmean})
    elif s == 'NHL':
        return pd.merge(cities, nhl_df, how='inner', left_on='Metropolitan area', right_on='location').groupby('Metropolitan area').agg({'W/L':np.nanmean,'Population':np.nanmean})
    elif s == 'MLB':
        return pd.merge(cities, mlb_df, how='inner', left_on='Metropolitan area', right_on='location').groupby('Metropolitan area').agg({'W/L':np.nanmean,'Population':np.nanmean})

def sports_team_performance():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for i in sports:
        m1 = sport_merger(i)
        for j in sports:
            if j != i:
                m2 = pd.merge(m1, sport_merger(j), how='inner', on='Metropolitan area')
                p_values.loc[i,j] = stats.ttest_rel(m2['W/L_x'], m2['W/L_y'])[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values